## Exercise 5: Nonlinear optimization, camera calibration.


These exercises will take you through:
- checkerboard calibration, in real life with OpenCV.
- non-linear optimization, where you will implement a function non-linear triangulation


In [34]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from scipy.spatial.transform import Rotation
from scipy.optimize import least_squares

### Nonlinear optimization

This exercise will take you through doing nonlinear optimization for triangulation of a single point. The same principles can be applied to more complex situations such as camera calibration, or situations where we lack a linear algoritm.

Construct two cameras with $R_1 = R_2 = I, t_1=\begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T,  t_2=\begin{bmatrix} 0 & 0 & 20 \end{bmatrix}^T$ and 

$$ K_1=K_2=\begin{bmatrix} f & \beta f & \delta_x \\ 0 & \alpha f & \delta_y \\ 0 & 0 & 1 \end{bmatrix}=\begin{bmatrix} 700 & 0 & 600 \\ 0 & 700 & 400 \\ 0 & 0 & 1 \end{bmatrix}$$

The cameras both observe the same 3D point $Q =\begin{bmatrix} 1 & 1 & 0 \end{bmatrix}^T$.




- **Exercise 5.1**. What are the projection matrices $P_1$ and $P_2$ ?.
What is the projection of $Q$ in cameras one and two ($q_1$ and $q_2$) ?


In [2]:
#Camera matrix
K=np.array([[700,0,600],[0,700,400],[0,0,1]])
R=np.eye(3) #rotation
t1=np.transpose([0,0,1]) #translation
t2=np.transpose([0,0,20]) #translation
Cam1=np.column_stack((R,t1))
Cam2=np.column_stack((R,t2))

#Projection matrices
P1=K@Cam1
P2=K@Cam2
print('The projection matrix of Camera 1:')
print(np.round(P1,2))
print('\n')
print('The projection matrix of Camera 2:')
print(np.round(P2,2))

The projection matrix of Camera 1:
[[700.   0. 600. 600.]
 [  0. 700. 400. 400.]
 [  0.   0.   1.   1.]]


The projection matrix of Camera 2:
[[7.0e+02 0.0e+00 6.0e+02 1.2e+04]
 [0.0e+00 7.0e+02 4.0e+02 8.0e+03]
 [0.0e+00 0.0e+00 1.0e+00 2.0e+01]]


In [3]:
def projectpoints(P,Q):
    [m,n]=np.shape(Q)
    cnt=np.ones(m)
    Q_ext=np.column_stack((Q,cnt))

    projection=P@Q_ext.T
    
    qx=np.transpose([projection[0]/projection[2]])
    qy=np.transpose([projection[1]/projection[2]])
    
    q=np.column_stack((qx,qy))
    
    return q

In [4]:
Q=np.array([[1,1,0]])

q1=projectpoints(P1,Q)
q2=projectpoints(P2,Q)

print('The projection of Q in camera 1 is:',q1)
print('The projection of Q in camera 2 is:',q2)

The projection of Q in camera 1 is: [[1300. 1100.]]
The projection of Q in camera 2 is: [[635. 435.]]


- **Exercise 5.2**.To simulate noise in the detection of points, we add errors to our projections.
$$q'_1 = q_1 +\begin{bmatrix} 1 & -1 \end{bmatrix}^T, q'_2 = q_2 + \begin{bmatrix} 1 & -1 \end{bmatrix}^T $$


Use your function `triangulate` from week 3 to triangulate $Q$ from $[q'_1,q'_2]$ and $[P_1, P_2]$.


Take the newly triangulated point $Q'$ and re-project it to the cameras. How far is it from our observations of the point ($q'_1$, $q'_2$)? 

In other words, what is the reprojection error for each camera? Is this as you expected when recalling the lecture from week 3?
How far is $Q'$ from $Q$?

In [5]:
def triangulate(qi,Pi):
    num=len(qi)
    B=[0,0,0,0]
    for a in range(num):
        x=qi[a][0][0]
        y=qi[a][0][1]
        P=Pi[a]
        #B is an array (2*n x 4), where n is the number of cameras
        Bi=np.array([P[2,:]*x-P[0,:],P[2,:]*y-P[1,:]])
        B=np.vstack([B, Bi])
        
    B=B[1:]
    u,s,vh= np.linalg.svd(B,full_matrices=True, compute_uv=True)
    
    BQ=vh[-1,:] / vh[-1,-1]
    Q_tri=np.array([[BQ[0],BQ[1],BQ[2]]])
    
    return Q_tri

In [7]:
Pi=[P1,P2]
q1_noise=q1+np.array([1,-1])
q2_noise=q2+np.array([1,-1])
qi_noise=[q1_noise,q2_noise]

Q_tri=triangulate(qi_noise,Pi)

print('The newly triangulated point Q is:',Q_tri)

The newly triangulated point Q is: [[1.01527507e+00 9.85270570e-01 2.85786810e-04]]


In [8]:
#We reproject the new Q_tri into the cameras:
q1_tri=projectpoints(P1,Q_tri)
q2_tri=projectpoints(P2,Q_tri)

qi_tri=[q1_tri,q2_tri]

In [9]:
#ERRORS:
#How far is Q from Q_tri
errorQ= (Q-Q_tri)**2
error_Q= np.sqrt(errorQ[0][0]+errorQ[0][1]+errorQ[0][2])
print('How far is Q_tri from Q:',error_Q)

#How far is q1 from q1_tri and q2 from q2_tri
error_q=[]
for i,a in enumerate(qi_tri):
    err=(qi_tri[i]-qi_noise[i])**2
    error_q.append(np.sqrt(err[0][0]+err[0][1]))
print('The reprojection error in camera 1 is:',error_q[0],'pixels.')
print('The reprojection error in camera 2 is:',error_q[1],'pixels.')


How far is Q_tri from Q: 0.021221817353380967
The reprojection error in camera 1 is: 13.433018988192183 pixels.
The reprojection error in camera 2 is: 0.6717725840473774 pixels.


<font color='darkblue'> We expect the linear algorithm to place a larger weight on the error of camera 2 than camera 1, as it has a larger s. Therefore camera 2 having the smallest reprojection error is as we expected.

- **Exercise 5.3**.We are going to make a new function `triangulate_nonlin` that does triangulation using nonlinear optimization. It should take the same inputs as `triangulate`, i.e. a list of n pixel coordinates $(q_1, q_2, ..., q_n)$, and a list of n projection matrices $(P_1, P_2, ..., P_n)$.


Start by defining a helper-function inside triangulate_nonlin.
This function, called `compute_residuals`, should take the parameters we want to optimize (in this case $Q$) as input, and should returns a vector of residuals (i.e. the numbers that we want to minimize the sum of squares of). In this case the residuals are the differences in projection, i.e.

$$\begin{bmatrix} \pi(P_1Q) - q'_1 \\ ... \end{bmatrix}$$

Call `triangulate` inside your function to get an initial guess `x0` and use `scipy.optimize.least_squares(compute_residuals, x0)`
to do least squares optimization, starting from the initial guess of your linear algorithm.

In [179]:
def compute_residuals(Q,qi_noise,Pi):
    Q=np.array(Q)
    return np.sqrt(np.sum((projectpoints(Pi[0],Q) - qi_noise[0])**2))**2 + np.sqrt(np.sum((projectpoints(Pi[1],Q) - qi_noise[1])**2))



In [180]:
def triangulate_nonlin(qi_noise,Pi):
    
    x_ini=triangulate(qi_noise,Pi) #initial guess
    BQ=least_squares(compute_residuals, x_ini[0], args=(qi_noise, Pi))
    
    return BQ


### Camera calibration with OpenCV

array([[1., 1.]])